In [1]:
import pandas as pd
import numpy as np
import glob, sys, os, time , itertools , warnings
from bs4 import BeautifulSoup
from tqdm import tqdm
import multiprocessing as mp
# from DataProcessing import 
import uszipcode

warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
path_to_csv = '/home/vlad/csv/Cars/'
path_to_output ='/home/vlad/csv/SC-output/'
colummn_names = ['Description'
                 ,'Title State/Type'
                 ,'Location'
                 ,'null'
                 ,'Auction Date'
                 ,'Actual Cash Value'
                 ,'Repair Cost'
                 ,'Odometer'
                 ,'Prim Damage'
                 ,'Sec Damage'
                 ,'Price Sold or Highest Bid']
path_to_makes = glob.glob(path_to_csv+'*')
search = uszipcode.SearchEngine(simple_zipcode=True)
path_to_all_years = list(itertools.chain.from_iterable([glob.glob(path+'/*') for path in path_to_makes]))
search = uszipcode.SearchEngine(simple_zipcode=True)

In [3]:
def process_files(path_to_years__):
    res_df = pd.DataFrame()
    #for each make/year folder get a list of all .html files
    #for path_to__year in tqdm(path_to_years__):
    for path_to__year in path_to_years__:
        try:
            #list of all html
            path_to_make_years = glob.glob(path_to__year+'/*')

            #for each .html in the folder
            for file_path in path_to_make_years:
            #file_path = path_to_make_years[0]
            
                #read file and load to the BS object
                with open(file_path, 'r') as f:
                    fle = f.read()
                soup = BeautifulSoup(fle)

                rows = []
                line_ind = 0 
                
                #filter all div elements (div table)
                for div in soup.find_all('div',['row']):
                    row = div.text.strip()
                    #print(row)
                    try:
                        line_dict = {}
                        i = 0
                        
                        #clean each element and map to respected column
                        for line in row.replace('\t','').replace('\xa0','').split('\n'):
                            line_dict[colummn_names[i]]= line.replace('Location:','').replace('Title State/Type:','').strip()
                            i+=1
                        rows.append(line_dict)
                    except Exception as e:
                        #print(line_ind,'-',e)
                        pass
                    finally:
                        line_ind+=1

                page_df = pd.DataFrame(rows).iloc[3:,:-1]
                page_df_ext = page_df.join(page_df['Description'].str.split(' ',expand=True).rename(columns={0:'Year',1:'Make',2:'Model',3:'Model 2',4:'Model 3'}))
                page_df_ext = page_df_ext.join(page_df_ext['Location'].str.split('-',expand=True).rename(columns={0:'State',1:'City'}))
                page_df_ext['City'] = page_df_ext['City'].str.strip().str.capitalize()
                page_df_ext['State'] = page_df_ext['State'].str.strip()
                res_df = res_df.append(page_df_ext,ignore_index=True,sort=False)
                del rows , page_df_ext , page_df
        except:
            continue
    return res_df

In [ ]:
%%time

if __name__ == '__main__':
    NCPU = mp.cpu_count()
    print('Creating pool with ', NCPU,' CPUs')
    pool = mp.Pool(NCPU)
    if not os.path.exists(path_to_output):
        print('Creating output folder')
        os.makedirs(path_to_output)
    
    by_chunk_path_to_all_years = np.array_split(path_to_all_years,len(path_to_all_years) // 4 )
    start_ = 0
    end_ = len(by_chunk_path_to_all_years)
    for i in range(start_,end_):
        print('Starting with chunk: ',i,' of ',end_)
        path_to_all_years_split = np.array_split(by_chunk_path_to_all_years[i],NCPU)
        ress = pool.map(process_files,path_to_all_years_split)
        ress_df = pd.concat(ress)
        print('Chunk ',i,' is done.')
        ress_df.to_csv(path_to_output+'sc_'+str(i)+'.csv',index=False)
#         i+=1
#         if i ==2:
#             break
    pool.close()
    pool.join()
    

In [4]:
outputs = glob.glob(path_to_output+'*')
out_l = []
for output in tqdm(outputs):
    out_l.append(pd.read_csv(output))
out_df = pd.concat(out_l,ignore_index=True,sort=False)

100%|██████████| 321/321 [00:08<00:00, 36.31it/s]


In [5]:


out_df = out_df.join(out_df['Location'].str.split('-',expand=True).rename(columns={0:'State',1:'City'}))

# out_df['City'] = out_df['City'].str.strip().str.capitalize()
out_df['State'] = out_df['State'].str.strip().str.upper()


out_df.head(20)

,Actual Cash Value,Auction Date,Description,Location,Odometer,Prim Damage,Repair Cost,Title State/Type,Make,Model,Model 2,Price Sold or Highest Bid,Sec Damage,Year,Model 3,5,State,City,2
0,NaN,11/05/2019,2005 Wabash DRY VAN,IL - CHICAGO NORTH,0 E,Minor dent/scratches,NaN,IL CT,Wabash,DRY,VAN,"$ 3,900 USD",NaN,2005,NaN,NaN,IL,CHICAGO NORTH,None
1,"$ 8,903 USD",10/23/2019,2005 Wabash DRY VAN,MA - WEST WARREN,0 E,Top/roof,NaN,PA SC,Wabash,DRY,VAN,$ 250 USD,NaN,2005,NaN,NaN,MA,WEST WARREN,None
2,NaN,08/27/2019,2005 Wabash DRY VAN,IN - INDIANAPOLIS,0 E,Front end,NaN,IN CT,Wabash,DRY,VAN,"$ 1,400 USD",NaN,2005,NaN,NaN,IN,INDIANAPOLIS,None
3,NaN,04/11/2019,2005 Wabash DRY VAN,IL - CHICAGO NORTH,0 E,Side,NaN,IL CT,Wabash,DRY,VAN,"$ 6,500 USD",NaN,2005,NaN,NaN,IL,CHICAGO NORTH,None
4,NaN,04/11/2019,2005 Wabash DRY VAN,IL - CHICAGO NORTH,0 E,Side,NaN,IL CT,Wabash,DRY,VAN,"$ 7,500 USD",NaN,2005,NaN,NaN,IL,CHICAGO NORTH,None
5,NaN,03/06/2019,2007 Wabash DRY VAN,IL - CHICAGO NORTH,0 E,Minor dent/scratches,NaN,IL CT,Wabash,DRY,VAN,"$ 7,000 USD",NaN,2007,NaN,NaN,IL,CHICAGO NORTH,None
6,"$ 14,862 USD",04/29/2019,2010 Wabash DRY VAN,GA - ATLANTA EAST,0 N,Rollover,NaN,GA BP,Wabash,DRY,VAN,"$ 1,600 USD",NaN,2010,NaN,NaN,GA,ATLANTA EAST,None
7,NaN,08/07/2019,2013 Wabash DRY VAN,CA - ANTELOPE,0 N,Minor dent/scratches,NaN,TN SC,Wabash,DRY,VAN,"$ 4,800 USD",NaN,2013,NaN,NaN,CA,ANTELOPE,None
8,"$ 5,725 USD",07/01/2016,2011 Chevrolet HHR LS,MO - ST. LOUIS,146669 A,FRONT END,"$ 5,540.83 USD",MO SC,Chevrolet,HHR,LS,"$ 1,000 USD",MINOR DENT/SCRATCHES,2011,NaN,NaN,MO,ST. LOUIS,None
9,"$ 13,125.04 USD",06/30/2016,2011 Chevrolet CAMARO LT,NM - ALBUQUERQUE,95843 A,FRONT END,"$ 12,364.94 USD",UT SC,Chevrolet,CAMARO,LT,"$ 4,500 USD",NaN,2011,NaN,NaN,NM,ALBUQUERQUE,None


In [6]:
out_df.groupby('Make')['Year'].count().sort_values(ascending=False)[:60]

Make
Toyota           142641
Honda            110234
Ford             100021
Chevrolet         95540
Nissan            90862
Hyundai           44489
Dodge             41854
BMW               40111
Mercedes-Benz     39617
Jeep              39128
Volkswagen        36164
KIA               29920
Lexus             29696
Chrysler          27465
Subaru            20847
Acura             16863
Audi              15197
Infiniti          15116
Pontiac           14658
Cadillac          13998
GMC               13314
Mitsubishi        12733
Volvo              9654
Lincoln            9611
Saturn             9465
Scion              7642
Land               7369
Mercury            7116
Mini               5623
Jaguar             4818
Porsche            4105
Mazda              2320
Buick              2300
Oldsmobile         2144
Saab               2089
Isuzu              1819
Fiat               1388
Hummer             1231
Maserati            774
Plymouth            676
Tesla               634
Smart      

In [13]:
out_df.shape

(1073859, 19)

In [ ]:
i = 6
city = out_df.loc[i,'City']
state =out_df.loc[i,'State']
city, state

geo_found = search.by_city_and_state(city,state)[0]

geo_found.county , geo_found.state

In [57]:

def find_county(x):
    try:
        return search.by_city_and_state(x[1] , x[0])[0].county
    except:
        return 'None'


locs = out_df['Location'].unique()
locs = locs[~pd.isnull(locs)]

state_city = [l[:2] for l in  list(map(lambda x: x.split(' - '),   locs ) ) #if len(l)>1
             ]





county = list(map(find_county,   state_city ) )

county = [', '.join([l.strip() for l in (county[i].replace('County','')+', '+state_city[i][0]).split(',') ])
 for i in range(len(county))]

loc_lookup = pd.DataFrame({'Location':locs,'County':county}).set_index('Location')['County']

In [113]:
loc_lookup

Location
IL - CHICAGO NORTH                                Lake, IL
MA - WEST WARREN                                  None, MA
IN - INDIANAPOLIS                               Marion, IN
GA - ATLANTA EAST                               Fulton, GA
CA - ANTELOPE                               Sacramento, CA
                                           ...            
CA - HAYWARD - DESERT VIEW                     Alameda, CA
IA - CRASHEDTOYS ELDRIDGE                        Scott, IA
MN - CRASHEDTOYS EAST BETHEL                     Anoka, MN
MN - CRASHEDTOYS MINNEAPOLIS                  Hennepin, MN
COPART AUTO AUCTIONS            None, COPART AUTO AUCTIONS
Name: County, Length: 206, dtype: object

In [102]:
%%time
def map_countries(x):
    try:
        return loc_lookup.loc[x]
    except:
        return 'None'
        
out_df['County'] = out_df['Location'].map(map_countries)

CPU times: user 44.5 s, sys: 67.5 ms, total: 44.5 s
Wall time: 44.6 s


In [103]:
out_df.to_csv(path_to_output+'SC-countymap.csv',index=False)

In [108]:
out_df['Prim Damage'].str.lower().unique()

array(['minor dent/scratches', 'top/roof', 'front end', 'side',
       'rollover', 'rear end', 'vandalism', 'water/flood', 'all over',
       'normal wear', 'undercarriage', 'biohazard/chemical', 'hail',
       'mechanical', 'partial repair', 'rejected repair', 'frame damage',
       'stripped', 'damage history', 'burn', 'burn - engine',
       'burn - interior', 'partial/incomplete r', 'frame damage reporte',
       'minor dents/scratche', 'biohazardous/chemica', 'replaced vin',
       'unknown', 'missing/altered vin', 'previous repair', nan,
       'price sold or highest bid', 'cash for clunkers', 'rr'],
      dtype=object)

In [109]:
out_df.describe()

,Actual Cash Value,Auction Date,Description,Location,Odometer,Prim Damage,Repair Cost,Title State/Type,Make,Model,Model 2,Price Sold or Highest Bid,Sec Damage,Year,Model 3,5,State,City,2,County
count,841322,1073857,1073859,1073820,1073857,1073853,389622,1073856,1073856,1073807,817211,1073856,468751,1073859,47793,265,1073820,1073619,6792,1073859
unique,80067,2621,26489,206,399918,79,100125,1387,90,2025,1141,901,77,101,210,13,61,196,2,165
top,"$ 8,500 USD",01/08/2019,2015 Nissan ALTIMA 2.5,TX - HOUSTON,0 N,Front end,"$ 10,000 USD",CA SC,Toyota,CAMRY,SE,"$ 1,500 USD",Minor dent/scratches,2007,S,LU,CA,HOUSTON,SUBLOT,"Los Angeles, CA"
freq,6995,1506,2217,45330,88499,250995,1014,103584,142641,39868,49903,27571,65669,77934,4567,90,192420,45330,6774,68045


In [112]:
from DataProcessing.DataStats import get_df_stats

get_df_stats(out_df)

,Feature,Unique_values,Percentage of missing values,Percentage of values in the biggest category,dtype
15,5,13,99.975323,99.975323,object
18,2,3,99.367515,99.367515,object
14,Model 3,210,95.549416,95.549416,object
6,Repair Cost,100125,63.717583,63.717583,object
12,Sec Damage,77,56.348925,56.348925,object
10,Model 2,1141,23.899599,23.899599,object
0,Actual Cash Value,80067,21.654333,21.654333,object
17,City,197,0.022349,4.221225,object
9,Model,2025,0.004842,3.712592,object
3,Location,206,0.003632,4.221225,object


In [106]:
out_df.head(40)

,Actual Cash Value,Auction Date,Description,Location,Odometer,Prim Damage,Repair Cost,Title State/Type,Make,Model,Model 2,Price Sold or Highest Bid,Sec Damage,Year,Model 3,5,State,City,2,County
0,NaN,11/05/2019,2005 Wabash DRY VAN,IL - CHICAGO NORTH,0 E,Minor dent/scratches,NaN,IL CT,Wabash,DRY,VAN,"$ 3,900 USD",NaN,2005,NaN,NaN,IL,CHICAGO NORTH,None,"Lake, IL"
1,"$ 8,903 USD",10/23/2019,2005 Wabash DRY VAN,MA - WEST WARREN,0 E,Top/roof,NaN,PA SC,Wabash,DRY,VAN,$ 250 USD,NaN,2005,NaN,NaN,MA,WEST WARREN,None,"None, MA"
2,NaN,08/27/2019,2005 Wabash DRY VAN,IN - INDIANAPOLIS,0 E,Front end,NaN,IN CT,Wabash,DRY,VAN,"$ 1,400 USD",NaN,2005,NaN,NaN,IN,INDIANAPOLIS,None,"Marion, IN"
3,NaN,04/11/2019,2005 Wabash DRY VAN,IL - CHICAGO NORTH,0 E,Side,NaN,IL CT,Wabash,DRY,VAN,"$ 6,500 USD",NaN,2005,NaN,NaN,IL,CHICAGO NORTH,None,"Lake, IL"
4,NaN,04/11/2019,2005 Wabash DRY VAN,IL - CHICAGO NORTH,0 E,Side,NaN,IL CT,Wabash,DRY,VAN,"$ 7,500 USD",NaN,2005,NaN,NaN,IL,CHICAGO NORTH,None,"Lake, IL"
5,NaN,03/06/2019,2007 Wabash DRY VAN,IL - CHICAGO NORTH,0 E,Minor dent/scratches,NaN,IL CT,Wabash,DRY,VAN,"$ 7,000 USD",NaN,2007,NaN,NaN,IL,CHICAGO NORTH,None,"Lake, IL"
6,"$ 14,862 USD",04/29/2019,2010 Wabash DRY VAN,GA - ATLANTA EAST,0 N,Rollover,NaN,GA BP,Wabash,DRY,VAN,"$ 1,600 USD",NaN,2010,NaN,NaN,GA,ATLANTA EAST,None,"Fulton, GA"
7,NaN,08/07/2019,2013 Wabash DRY VAN,CA - ANTELOPE,0 N,Minor dent/scratches,NaN,TN SC,Wabash,DRY,VAN,"$ 4,800 USD",NaN,2013,NaN,NaN,CA,ANTELOPE,None,"Sacramento, CA"
8,"$ 5,725 USD",07/01/2016,2011 Chevrolet HHR LS,MO - ST. LOUIS,146669 A,FRONT END,"$ 5,540.83 USD",MO SC,Chevrolet,HHR,LS,"$ 1,000 USD",MINOR DENT/SCRATCHES,2011,NaN,NaN,MO,ST. LOUIS,None,"St. Charles, MO"
9,"$ 13,125.04 USD",06/30/2016,2011 Chevrolet CAMARO LT,NM - ALBUQUERQUE,95843 A,FRONT END,"$ 12,364.94 USD",UT SC,Chevrolet,CAMARO,LT,"$ 4,500 USD",NaN,2011,NaN,NaN,NM,ALBUQUERQUE,None,"Bernalillo, NM"


In [105]:
# import pickle
# with open(path_to_output+'SC-countymap.pkl', 'wb') as f:
#     pickle.p(f,out_df)